In [36]:
from sklift.datasets import fetch_x5
from sklearn.model_selection import train_test_split
from catboost import CatBoost 
import lightgbm as LGBm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# load date

In [25]:
dataset = fetch_x5()
data, target, treatment = dataset.data, dataset.target, dataset.treatment

# data - dictionary-like object
# data contains general info about clients:
clients = data.clients

# data contains a subset of clients for training:
train = data.train

# data contains a clients’ purchase history prior to communication.
purchases = data.purchases

# research date

In [26]:
clients.head()

,client_id,first_issue_date,first_redeem_date,age,gender
0,000012768d,2017-08-05 15:40:48,2018-01-04 19:30:07,45,U
1,000036f903,2017-04-10 13:54:23,2017-04-23 12:37:56,72,F
2,000048b7a6,2018-12-15 13:33:11,NaN,68,F
3,000073194a,2017-05-23 12:56:14,2017-11-24 11:18:01,60,F
4,00007c7133,2017-05-22 16:17:08,2018-12-31 17:17:33,67,U


In [27]:
train.head()

,client_id
0,000012768d
1,000036f903
2,00010925a5
3,0001f552b0
4,00020e7b18


In [28]:
purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45786568 entries, 0 to 45786567
Data columns (total 13 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   client_id                object 
 1   transaction_id           object 
 2   transaction_datetime     object 
 3   regular_points_received  float64
 4   express_points_received  float64
 5   regular_points_spent     float64
 6   express_points_spent     float64
 7   purchase_sum             float64
 8   store_id                 object 
 9   product_id               object 
 10  product_quantity         float64
 11  trn_sum_from_iss         float64
 12  trn_sum_from_red         float64
dtypes: float64(8), object(5)
memory usage: 4.4+ GB


In [29]:
target.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [30]:
treatment.head()

0    0
1    1
2    1
3    1
4    1
Name: treatment_flg, dtype: int64

In [31]:
print(clients.shape, train.shape, target.shape, treatment.shape, purchases.shape)

(400162, 5) (200039, 1) (200039,) (200039,) (45786568, 13)


# study

In [32]:
use_cols = [
    'client_id',
    'purchase_sum',
    'trn_sum_from_iss',
    'trn_sum_from_red',
    'regular_points_received',
    'express_points_received',
    'regular_points_spent',
    'express_points_spent',
    'product_quantity',
]

dtypes = {
    'client_id': 'object',
    'purchase_sum': 'float32',
    'trn_sum_from_iss': 'float32',
    'trn_sum_from_red': 'float32',
    'regular_points_received': 'float32',
    'express_points_received': 'float32',
    'regular_points_spent': 'float32',
    'express_points_spent': 'float32',
    'product_quantity': 'float32',
}


In [35]:
unic_clients = clients["client_id"]
split_client = np.array_split(unic_clients, 20)
agg_clients = []

for ids in split_client:
    chunk = purchases[purchases["client_id"].isin(ids)]

    g = chunk.groupby("client_id").agg(
        txn_cnt=('purchase_sum', 'size'),

        purchase_sum_total=('purchase_sum', 'sum'),
        purchase_sum_mean=('purchase_sum', 'mean'),
        purchase_sum_std=('purchase_sum', 'std'),
        purchase_sum_max=('purchase_sum', 'max'),

        iss_sum=('trn_sum_from_iss', 'sum'),
        red_sum=('trn_sum_from_red', 'sum'),

        regular_points_received_sum=('regular_points_received', 'sum'),
        express_points_received_sum=('express_points_received', 'sum'),

        regular_points_spent_sum=('regular_points_spent', 'sum'),
        express_points_spent_sum=('express_points_spent', 'sum'),

        product_quantity_total=('product_quantity', 'sum'),
    )

    g['points_received_total'] = (
        g['regular_points_received_sum'] + g['express_points_received_sum']
    )

    g['points_spent_total'] = (
        g['regular_points_spent_sum'] + g['express_points_spent_sum']
    )

    agg_clients.append(g)

train_after_agg = pd.concat(agg_clients).groupby(level=0).sum().reset_index()